In [9]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.layers import Input
from keras.utils import np_utils
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D

import cv2
import numpy as np

In [2]:
catmapfilename='breed_map.list'
catmapfile = open(catmapfilename,'r')
categoryMap = dict()
for line in catmapfile.readlines():
    ls = line.split()
    category = ls[0]
    breed = ls[1]
    categoryMap[category] = breed

categoryIndex = dict(zip(categoryMap.keys(),range(len(categoryMap))))
nb_classes = len(categoryMap)

In [3]:
input_photos = []
breedlist = []
posfilename = "positives_mod10.txt"
inputdirname = "CroppedDogs/"
posfile = open(posfilename,'r')
for line in posfile.readlines():
    infile = line.strip()
    ls0 = infile.split("/")
    ls1 = ls0[1].split("_")
    category = ls1[0]
    img = cv2.resize(cv2.imread(infile), (299, 299), cv2.INTER_LINEAR)
    input_photos.append(img)
    breedlist.append(categoryIndex[category])

In [13]:
print len(input_photos),len(breedlist)
batch_size = 10
nb_epoch = 50

2212 2212


In [5]:
train = np.array(input_photos)
labels = np.array(breedlist)

print train.shape

nToUse = len(train)
testFraction = 1.0/6.0
nTest = int(testFraction*nToUse)

X_test = train[:nTest]
y_test = labels[:nTest]

X_train = train[nTest:]
y_train = labels[nTest:]

data_augmentation = False

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

# input image dimensions
img_rows, img_cols = X_train.shape[1], X_train.shape[2]

if K.image_dim_ordering() == 'th':
    X_train = np.swapaxes(X_train,1,3)
    X_train = np.swapaxes(X_train,2,3)
    X_test = np.swapaxes(X_test,1,3)
    X_test = np.swapaxes(X_test,2,3)
    input_shape = (train.shape[3], img_rows, img_cols)
else:
	input_shape = (img_rows, img_cols, train.shape[3])

print input_shape

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

(2212, 299, 299, 3)
(3, 299, 299)


In [10]:
# create the base pre-trained model

# this could also be the output a different Keras model or layer
#input_tensor = Input(shape=(3, 128, 128))  # this assumes K.image_dim_ordering() == 'tf'

model = Sequential()

model.add(Convolution2D(50, 5,5,border_mode='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Convolution2D(40, 3,3, border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(30, 2,2, border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(20, 2,2, border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(20, 2,2, border_mode='same'))
model.add(Activation('relu'))

#model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))


In [11]:
model.compile(optimizer='adadelta', loss='categorical_crossentropy',metrics=['accuracy'])

In [14]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_split=1/10.0)

Train on 1659 samples, validate on 185 samples
Epoch 1/50
1659/1659 [==============================] - 58s - loss: 4.5138e-06 - acc: 1.0000 - val_loss: 16.0796 - val_acc: 0.0000e+00

In [15]:
model.save("dog_cnn_simple.keras")

In [16]:
score = model.evaluate(X_test, Y_test, verbose=0)
Y_pred = model.predict(X_test)
print 'Test score:', score[0]
print 'Test accuracy:', score[1]


Test score: 16.1180953979
Test accuracy: 0.0
